In [39]:
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE, SVMSMOTE
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score,accuracy_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, GridSearchCV

In [2]:
ls

 application_test.csv             full.csv
 application_train.csv            HomeCredit_columns_description.csv
 bureau_balance.csv               installments_payments.csv
 bureau.csv                       LightGBM.ipynb
 credit_card_balance.csv         'Model Performance.ipynb'
 Data_Exploration_FullSet.ipynb   POS_CASH_balance.csv
 Data_Exploration.ipynb           previous_application.csv
 feat_eng.csv                     sample_submission.csv
 Feature_Eng.ipynb


In [3]:
dataset = pd.read_csv("feat_eng.csv")

In [4]:
dataset.head()

,Unnamed: 0,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,...,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_LIMIT_ACTUAL,AMT_CREDIT_y,AMT_CREDIT_x.1,AMT_CREDIT_y.1,AMT_CREDIT
0,0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,...,0.0,0.0,0.0,0.0,1.0,0.0,179055.00,179055.00,179055.00,179055.00
1,1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,...,0.0,0.0,0.0,0.0,0.0,0.0,484191.00,484191.00,484191.00,484191.00
2,2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,20106.00,20106.00,20106.00,20106.00
3,3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,...,NaN,NaN,NaN,NaN,NaN,270000.0,291695.50,291695.50,291695.50,291695.50
4,4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,166638.75,166638.75,166638.75,166638.75


In [5]:
dataset.shape

(307511, 128)

In [6]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Get  columns whose data type is object i.e. string
filteredColumns = dataset.dtypes[dataset.dtypes == np.object]
# list of columns whose data type is object i.e. string
listOfColumnNames = list(filteredColumns.index)
print(listOfColumnNames)

# Label Encoding if there are two categorical labels on a feature)
for col in listOfColumnNames:
    if dataset.dtypes[col] == np.object:
        # If 2 or fewer unique categories
        if len(list(dataset[col].unique())) <= 2:
            # Train on the training data
            le.fit(dataset[col])
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

# One-Hot Encoding (> 2 categorical labels for a feature)
num_dataset = pd.get_dummies(dataset)

['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']
3 columns were label encoded.


In [7]:
null_rows = num_dataset[num_dataset.isnull().any(axis=1)]
null_rows

,Unnamed: 0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,...,0,0,0,0,0,0,1,0,1,0
1,1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,...,0,1,0,0,0,0,0,0,1,0
2,2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,...,0,0,0,0,0,0,0,0,0,0
3,3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,...,0,0,0,0,0,0,0,0,0,0
4,4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,307506,456251,0,0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327,...,0,0,0,0,0,0,1,0,1,0
307507,307507,456252,0,0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775,...,0,0,0,0,0,0,1,0,1,0
307508,307508,456253,0,0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966,...,0,0,0,0,0,1,0,0,1,0
307509,307509,456254,1,0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961,...,0,0,0,0,0,0,1,0,1,0


In [8]:
null_rows.shape

(296892, 252)

What percent of the rows in the data have a null value in at least one column?

In [9]:
null_rows.shape[0]/dataset.shape[0]

0.9654679019612307

97% of the rows have at least one column with a null value. 
How will we handle null values?

In [10]:
dataset.columns

Index(['Unnamed: 0', 'SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE',
       'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'AMT_CREDIT_x',
       ...
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR', 'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_CREDIT_y',
       'AMT_CREDIT_x.1', 'AMT_CREDIT_y.1', 'AMT_CREDIT'],
      dtype='object', length=128)

In [11]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(num_dataset)
new_num_dataset = pd.DataFrame(imp.transform(num_dataset))
new_num_dataset.columns = num_dataset.columns
new_num_dataset.index = num_dataset.index
num_dataset = new_num_dataset

null_rows = num_dataset[num_dataset.isnull().any(axis=1)]
null_rows

,Unnamed: 0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes


In [12]:
num_dataset.shape

(307511, 252)

In [13]:
num_dataset.head()

,Unnamed: 0,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT_x,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,0.0,100002.0,1.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,100003.0,0.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2.0,100004.0,0.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,100006.0,0.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,100007.0,0.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [40]:
correls = {}
counter = -1
for col in num_dataset.columns:
    counter+=1
    c = num_dataset['TARGET'].corr(num_dataset[col])
    if c is not None:
        correls[col] = float(c)
    else:
        correls[col] = (None)

In [42]:
for w in sorted(correls, key=correls.get, reverse=True):
    print(w, correls[w])

TARGET 1.0
DAYS_BIRTH 0.07823930830984512
REGION_RATING_CLIENT_W_CITY 0.06089266756485737
REGION_RATING_CLIENT 0.05889901494571978
NAME_INCOME_TYPE_Working 0.0574811848183828
DAYS_LAST_PHONE_CHANGE 0.05521847562885774
CODE_GENDER_M 0.054712681971286585
DAYS_ID_PUBLISH 0.051457172607069564
REG_CITY_NOT_WORK_CITY 0.05099446436811681
NAME_EDUCATION_TYPE_Secondary / secondary special 0.04982373084813512
FLAG_EMP_PHONE 0.045982219716588515
REG_CITY_NOT_LIVE_CITY 0.044395374805692424
FLAG_DOCUMENT_3 0.04434634685113139
OCCUPATION_TYPE_Laborers 0.043018954955329355
DAYS_REGISTRATION 0.04197486283142708
LIVE_CITY_NOT_WORK_CITY 0.032518341101488414
DEF_30_CNT_SOCIAL_CIRCLE 0.032221533905931195
DEF_60_CNT_SOCIAL_CIRCLE 0.031251211111733374
NAME_CONTRACT_TYPE_Cash loans 0.0308959081610508
OCCUPATION_TYPE_Drivers 0.030303499914959744
NAME_HOUSING_TYPE_With parents 0.02996585666593896
ORGANIZATION_TYPE_Self-employed 0.0291388161708327
FLAG_WORK_PHONE 0.02852432236322825
OCCUPATION_TYPE_Low-skill La

In [14]:
X_train, X_test, y_train, y_test = train_test_split(num_dataset.drop('TARGET', 1), 
                                    num_dataset['TARGET'], test_size = .2, random_state=42,shuffle=True)

In [15]:
# SMOTE: Overclassing Minority Classification

sum_1 = sum(y_train==1)
sum_0 = sum(y_train==0)

print('Total Data', len(y_train))
print('Imbalance: [-]', sum_0, '[+]', sum_1)

print("\nPerforming SMOTE on training data set\n")
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

sum_1 = sum(y_train_res==1)
sum_0 = sum(y_train_res==0)

print('Total Data', len(y_train_res))
print('Imbalance: [-]', sum_0, '[+]', sum_1)

Total Data 246008
Imbalance: [-] 226132 [+] 19876

Performing SMOTE on training data set

Total Data 452264
Imbalance: [-] 226132 [+] 226132


In [16]:
#amount of rows in input data
X_train_res.shape[0]

452264

In [17]:
#amount of columns in input data
len(X_train_res.columns)

251

In [18]:
X_train_res['AMT_CREDIT_LIMIT_ACTUAL']

0         151875.00000
1              0.00000
2              0.00000
3              0.00000
4          93145.16129
              ...     
452259         0.00000
452260     90000.00000
452261         0.00000
452262         0.00000
452263         0.00000
Name: AMT_CREDIT_LIMIT_ACTUAL, Length: 452264, dtype: float64

In [19]:
model = DecisionTreeClassifier()
model.fit(X_train_res, y_train_res)
y_pred=model.predict(X_test)

In [20]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8488366421150187


In [21]:
print(confusion_matrix(y_test,y_pred))

[[51415  5139]
 [ 4158   791]]


In [22]:
print("Decision Tree TRAIN score with ",format(model.score(X_train_res, y_train_res)))
print("Decision Tree TEST score with ",format(model.score(X_test, y_test)))

Decision Tree TRAIN score with  1.0
Decision Tree TEST score with  0.8488366421150187


In [23]:
auc = roc_auc_score(y_test, y_pred)
auc

0.5344806823424298

In [24]:
num_dataset.to_csv("full.csv")

In [25]:
#cross_validate
cv_results = cross_validate(model, X_train_res, y_train_res, cv=10)

In [26]:
#score with cross validation
np.mean(cv_results['test_score'])

0.9079768213187801

In [27]:
param_grid = {"criterion": ['entropy', 'gini'],
"min_samples_split": [5,10,15],
"max_depth": [2,3,5],
"min_samples_leaf": [5,10,15],
"max_leaf_nodes": [5,10,15],
}
CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
CV_rfc.fit(X_train_res, y_train_res)
CV_rfc.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_leaf_nodes': 15,
 'min_samples_leaf': 5,
 'min_samples_split': 5}

In [28]:
model = DecisionTreeClassifier(criterion='entropy', max_depth=2, 
                               max_leaf_nodes=5, min_samples_leaf=5, min_samples_split=5)
model.fit(X_train_res, y_train_res)
y_pred=model.predict(X_test)

In [29]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6368307237045348


In [30]:
print("Decision Tree TRAIN score with ",format(model.score(X_train_res, y_train_res)))
print("Decision Tree TEST score with ",format(model.score(X_test, y_test)))

Decision Tree TRAIN score with  0.6618103585516424
Decision Tree TEST score with  0.6368307237045348


In [31]:
print(confusion_matrix(y_test,y_pred))

[[36540 20014]
 [ 2322  2627]]


In [32]:
auc = roc_auc_score(y_test, y_pred)
auc

0.588461225174361